<a href="https://colab.research.google.com/github/AiratGaliev/llama-cpp-server-colab/blob/main/llama-cpp-server-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Start the llama-cpp-server

#@markdown If unsure about the branch, write "main" or leave it blank.
%cd /content
!apt-get -y install -qq aria2
!npm install -g localtunnel
!export CUDACXX=/usr/local/cuda/bin/nvcc
!export CMAKE_ARGS="-DLLAMA_CUBLAS=on"
!export FORCE_CMAKE=1
!pip install -U pip setuptools wheel
!pip install llama-cpp-python[server]==0.2.38 --force-reinstall --upgrade --no-cache-dir --verbose

# Parameters
model_url = "TheBloke/dolphin-2.6-mistral-7B-dpo-laser-GGUF" #@param {type:"string"}
model_alias = "dolphin-2.6-mistral-7B-dpo-laser" #@param {type:"string"}
chat_format = "chatml" #@param {type:"string"}
n_gpu_layers = 33 #@param {type:"integer"}
offload_kqv = True, #@param {type:"boolean"}
n_threads = 12, #@param {type:"integer"}
n_batch = 512, #@param {type:"integer"}
n_ctx = 5120, #@param {type:"integer"}
cache = False #@param {type:"string"}

model_url = model_url.strip()
model = ""
if model_url != "":
    if not model_url.startswith('http'):
        model_url = f"https://huggingface.co/{model_url}/blob/main/{model_alias}.Q8_0.gguf"

# Download the model
models_dir = f"/content/models/"
download_cmd = f"aria2c --console-log-level=error -c -x 16 -s 16 -k 1M {model_url} -d {models_dir} -o {model_alias}.Q8_0.gguf"
print(download_cmd)
!$download_cmd

# Start the server
model = f"{models_dir}{model_alias}.Q8_0.gguf"
command_line_flags = ""
for param in ['model','model_alias','chat_format', 'n_gpu_layers', 'offload_kqv', 'n_threads', 'n_batch', 'n_ctx', 'cache']:
    command_line_flags += f" --{param} {param}"

cmd = f"python -m llama_cpp.server --port 1234 {command_line_flags}"
print(cmd)
!$cmd